In [7]:
import numpy as np
import pandas as pd
import zipfile
from zipfile import ZipFile
import glob
import os
from openpyxl import load_workbook
pd.options.display.float_format = '{:,.2f}'.format

# DATA RIESGO

In [11]:
data_risk = pd.read_excel('TXS del 24092022 duplicados.xlsx',converters={'Last4': str,'Folio':str,'Auth number':str,
                                                                         'Affiliation':str})
data_risk.columns = data_risk.columns.str.strip().str.lower() 
data_risk['last4'] = data_risk['last4'].str.zfill(4)
data_risk['INDEX'] = (data_risk['auth number'].astype(str).replace(' ', '',regex=True)+'_'
                       +data_risk['affiliation'].astype(str).replace(' ', '',regex=True)+'_'
                       +data_risk['last4'].astype(str).replace(' ', '',regex=True)+'_'
                       +data_risk['amount'].astype(str).replace(' ', '',regex=True))
data_risk.head()

,me created,admin name,me mail,txn date clip,amount,last4,ligavoucher,folio,transaction id,receipt no,auth number,affiliation,adquirente,INDEX
0,2021-11-17 14:48:50,Mario Alberto Merito Dela Rosa,3321821394mayito@gmail.com,2022-09-24 22:50:23,218.00,0619,https://receipt.clip.mx/f6f4f540-371e-4ab2-9a3...,2022101130988,f6f4f540-371e-4ab2-9a35-f7d720326f61,KeQ2RK4,241237,07927517,PROSA,241237_07927517_0619_218.0
1,2021-11-17 14:48:50,Mario Alberto Merito Dela Rosa,3321821394mayito@gmail.com,2022-09-24 00:00:00,218.00,0619,-,2022101130989,NaN,NaN,241237,07927517,NaN,241237_07927517_0619_218.0
2,2021-10-30 17:39:49,Karla Berenice Zaragoza Flores,49194karlazaragoza@gmail.com,2022-09-24 22:32:50,165.00,5577,https://receipt.clip.mx/6cce2224-04b4-4c9f-8c8...,2022101131255,6cce2224-04b4-4c9f-8c8b-99b4e91fdd54,BUUcwNm,225617,07927517,PROSA,225617_07927517_5577_165.0
3,2021-10-30 17:39:49,Karla Berenice Zaragoza Flores,49194karlazaragoza@gmail.com,2022-09-24 00:00:00,165.00,5577,-,2022101131256,NaN,NaN,225617,07927517,NaN,225617_07927517_5577_165.0
4,2019-04-02 17:45:26,Alfonso Morales Orozco,6621844608z@gmail.com,2022-09-24 22:09:59,37.00,4402,https://receipt.clip.mx/8630de4e-d9c8-4364-812...,2022101130482,8630de4e-d9c8-4364-812a-1bcdd870f0c9,M98GX43,203271,07927517,PROSA,203271_07927517_4402_37.0


In [12]:
data_risk.shape

(986, 14)

# DATA CRUCE

In [15]:
data_me = pd.read_excel('Transacciones_duplicadas_disputas.xlsx',converters={'last4': str,'codigo_autorizacion':str,'order_id':str,
                                                                         'order_id_xpay_x':str,'affiliation_sftp':str})
data_me.columns = data_me.columns.str.strip().str.lower() 
data_me['INDEX'] = (data_me['codigo_autorizacion'].astype(str).replace(' ', '',regex=True)+'_'
                       +data_me['affiliation_sftp'].astype(str).replace(' ', '',regex=True)+'_'
                       +data_me['last4'].astype(str).replace(' ', '',regex=True)+'_'
                       +data_me['gross_amount'].astype(str).replace(' ', '',regex=True))
data_me.head()

,merchant_id,uuid,transaction_id,order_id,affiliation,affiliation_type,sub_affiliation,mcc,date_transaction,created_at,...,order_id_busqueda,affiliation_sftp,transaction_date,transaction_type,index_total_y,order_id_xpay_y,codigo_autorizacion,deposit_date,gross_amount,INDEX
0,7e90cd0e-9dfd-45b2-95c9-ed6a9951a8e5,914b8587-7a42-4a16-85df-5987f06195f2,adadf401-2ab8-46c8-9518-1054dbdab594,660979800539,7927515,CARD_PRESENT,1725573,5912,2022-09-24,2022-09-24 16:38:54,...,"'660979800539',",07927515,2022-09-24,SETTLED,434258_32_660979800539_7813.0,NaN,054041,2022-09-26,"7,813.00",054041_07927515_1032_7813.0
1,01a94e53-3c0e-47e4-8786-661fd4975150,a0a67e2e-9a5c-44fb-9b7e-1f70cd899e2a,79bfa40d-c83a-4185-ab96-99568e1bdddb,784958477006,7927515,CARD_PRESENT,1322737,5912,2022-09-24,2022-09-24 18:52:17,...,"'784958477006',",07927515,2022-09-24,SETTLED,415231_17_784958477006_120.0,NaN,894024,2022-09-26,120.00,894024_07927515_5517_120.0
2,01a94e53-3c0e-47e4-8786-661fd4975150,cc33a215-792c-415b-af9a-27956887222c,f61cac28-4832-44ad-8e5c-52dd926d63ce,153443652016,7927515,CARD_PRESENT,1322737,5912,2022-09-24,2022-09-24 18:53:26,...,"'153443652016',",07927515,2022-09-24,SETTLED,415231_17_153443652016_120.0,NaN,911551,2022-09-26,120.00,911551_07927515_5517_120.0
3,5a67ab03-5938-41af-901c-52ca093b71ed,952ff096-bd4c-450d-bae5-a65898a848eb,bbfe366d-a198-4ff9-86b9-01d159e9d102,194329179709,7927500,CARD_PRESENT,1473184,5499,2022-09-24,2022-09-24 20:46:30,...,"'194329179709',",07927500,2022-09-24,SETTLED,525678_91_194329179709_330.0,NaN,006444,2022-09-26,330.00,006444_07927500_5591_330.0
4,5a67ab03-5938-41af-901c-52ca093b71ed,390f102b-0e58-4c97-98bf-98fc83dd31d0,7612d855-d69e-4a6c-adc6-7bffceaed042,752305395839,7927500,CARD_PRESENT,1473184,5499,2022-09-24,2022-09-24 20:47:22,...,"'752305395839',",07927500,2022-09-24,SETTLED,525678_91_752305395839_330.0,NaN,013857,2022-09-26,330.00,013857_07927500_5591_330.0


In [16]:
data_me.shape

(1958, 37)

# CRUCE

In [17]:
match = data_risk.merge(data_me[['INDEX','order_id','order_id_xpay_y','transaction_id','transaction_date',
                                          'status_code','transaction_type','deposit_date','gross_amount']],on='INDEX',
                               how='left')
match.head()

,me created,admin name,me mail,txn date clip,amount,last4,ligavoucher,folio,transaction id,receipt no,...,adquirente,INDEX,order_id,order_id_xpay_y,transaction_id,transaction_date,status_code,transaction_type,deposit_date,gross_amount
0,2021-11-17 14:48:50,Mario Alberto Merito Dela Rosa,3321821394mayito@gmail.com,2022-09-24 22:50:23,218.00,0619,https://receipt.clip.mx/f6f4f540-371e-4ab2-9a3...,2022101130988,f6f4f540-371e-4ab2-9a35-f7d720326f61,KeQ2RK4,...,PROSA,241237_07927517_0619_218.0,154478322689,NaN,f6f4f540-371e-4ab2-9a35-f7d720326f61,2022-09-24,1.00,SETTLED,2022-09-26,218.00
1,2021-11-17 14:48:50,Mario Alberto Merito Dela Rosa,3321821394mayito@gmail.com,2022-09-24 22:50:23,218.00,0619,https://receipt.clip.mx/f6f4f540-371e-4ab2-9a3...,2022101130988,f6f4f540-371e-4ab2-9a35-f7d720326f61,KeQ2RK4,...,PROSA,241237_07927517_0619_218.0,154478322689,NaN,f6f4f540-371e-4ab2-9a35-f7d720326f61,2022-09-24,1.00,SETTLED,2022-09-28,218.00
2,2021-11-17 14:48:50,Mario Alberto Merito Dela Rosa,3321821394mayito@gmail.com,2022-09-24 22:50:23,218.00,0619,https://receipt.clip.mx/f6f4f540-371e-4ab2-9a3...,2022101130988,f6f4f540-371e-4ab2-9a35-f7d720326f61,KeQ2RK4,...,PROSA,241237_07927517_0619_218.0,154478322689,NaN,f6f4f540-371e-4ab2-9a35-f7d720326f61,2022-09-24,1.00,SETTLED,2022-09-26,218.00
3,2021-11-17 14:48:50,Mario Alberto Merito Dela Rosa,3321821394mayito@gmail.com,2022-09-24 22:50:23,218.00,0619,https://receipt.clip.mx/f6f4f540-371e-4ab2-9a3...,2022101130988,f6f4f540-371e-4ab2-9a35-f7d720326f61,KeQ2RK4,...,PROSA,241237_07927517_0619_218.0,154478322689,NaN,f6f4f540-371e-4ab2-9a35-f7d720326f61,2022-09-24,1.00,SETTLED,2022-09-28,218.00
4,2021-11-17 14:48:50,Mario Alberto Merito Dela Rosa,3321821394mayito@gmail.com,2022-09-24 00:00:00,218.00,0619,-,2022101130989,NaN,NaN,...,NaN,241237_07927517_0619_218.0,154478322689,NaN,f6f4f540-371e-4ab2-9a35-f7d720326f61,2022-09-24,1.00,SETTLED,2022-09-26,218.00


In [18]:
match.shape

(3903, 22)

In [19]:
match.to_excel('DISPUTAS BNTE 2022.10.12-V1.xlsx',index=False)